# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
import json
import os
import logging
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

level = logging.WARNING

logger = logging.getLogger(__name__)
logging.basicConfig(level=level)
logger.setLevel(level)
logging.getLogger("llama_index").setLevel(level)
logging.getLogger("asyncio").setLevel(level)

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY")); openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b
multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b
add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [4]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import (
    ExperimentalOpenAIAgent
)
from llama_index.llms import OpenAI

In [5]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True
)

### Chat

In [6]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405
(121 * 3) + 42 is equal to 405.


In [7]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [8]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 * 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [9]:
response = exp_agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a poem about a group of mice."
)

response_gen = response.response_gen

for ix, token in enumerate(response_gen):
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
121 * 2 is equal to 242.

In a cozy corner, where shadows dance,
A group of mice embark on a grand romance.
With tiny paws and whiskers so fine,
They scurry and scamper, in perfect rhyme.

Their fur, a tapestry of colors so bright,
They explore the world, both day and night.
With nimble feet, they traverse the floor,
Seeking adventure, forever wanting more.

In fields of cheese, their treasure trove,
They nibble and feast, their hunger drove.
With tiny squeaks, they sing their song,
A chorus of joy, all night long.

They build their nests, with twigs and straw,
A cozy haven, where dreams can thaw.
In unity, they stand, a family so tight,
Facing the world, with all their might.

Oh, little mice, with hearts so pure,
Your spirit and courage, forever endure.
In this vast world, you find your place,
A reminder of resilience and grace.

So let us celebrate these creatures small,
For they 

### Async Streaming Chat

In [10]:
response = await exp_agent.astream_chat(
    "What is 121 * 2 + 8? Once you have the answer, use that number to write a limerick about a group of mice."
)

async for token in response.async_response_gen():
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
=== Calling Function ===
Calling function: add with args: {
  "a": 242,
  "b": 8
}
Got output: 250
121 * 2 + 8 is equal to 250.

There once was a group of mice,
Living in a world so nice.
With fur so soft and tails so long,
They scurried and sang a merry song.

In fields of cheese, they would play,
Having fun throughout the day.
With tiny paws and nimble feet,
They danced and laughed, oh so sweet.

Their leader, a mouse named Mike,
Was clever and quick, never one to strike.
He guided his friends with wisdom and grace,
In their cozy little mousey space.

Together they explored, without a care,
In their tiny world, beyond compare.
With hearts so full and spirits high,
The mice lived happily, under the sky.

So let us raise a toast, to this charming crew,
The mice who bring joy, it's true.
In their limerick tale, we find delight,
A reminder to cherish each moment, day and night.

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)